# 모델 평가지표

PPT 참고

In [102]:
y_pred = [[0.99, 0, 0.01, 0, 0],
[0.2, 0.1, 0.3, 0.2 , 0.2],
[0.1, 0.6, 0.1, 0.1, 0.1],
[0.5, 0.2, 0.1, 0.1, 0.1]]

In [103]:
y_true = [[1,0,0,0,0],
[0,0,1,0,0],
[0,1,0,0,0],
[0,0,0,0,1]]

In [104]:
from sklearn.metrics import log_loss

In [105]:
log_loss(y_true, y_pred)

1.0068584642348692

In [107]:
from math import log

In [108]:
import numpy as np

In [109]:
-(log(0.99) + log(0.3) + log(0.6) + log(0.1)) / 4

1.0068584642348684

# 검증셋 구축

CV 스코어랑 LB 스코어가 어떻게 되시나요??
- cross validation score A변화를 줬을 때, CV 올라갔따, LB 올라갔다 (이상적인) -> CV 내려갈 때, LB도 내려가는 경우 
- leaderboard score 
- CV가 올라갈 때, LB가 내려가는, CV가 내려갈 때 LB가 올라가는 

In [10]:
import os
import pandas as pd 
import numpy as np
import multiprocessing # 여러 개의 일꾼 (cpu)들에게 작업을 분산시키는 역할
from multiprocessing import Pool 
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할
from data_loader_v2 import data_loader_v2 # 자체적으로 만든 data loader version 2.0 ([데이콘 15회 대회] 데이터 설명 및 데이터 불러오기 영상 참조)
from tqdm import tqdm

In [12]:
train_folder = '../train/'
test_folder = '../test/'
train_label_path = '../etc/train_label.csv'

train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [13]:
# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(tqdm(pool.imap(func_fixed, files), total = len(files)))
        pool.close()
        pool.join()       
    combined_df = pd.concat(df_list)    
    return combined_df

## 데이터 불러오기

In [14]:
train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, event_time=10, nrows=40)

100%|██████████| 826/826 [01:47<00:00,  7.66it/s]


In [15]:
test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=10, nrows=40)

100%|██████████| 718/718 [01:33<00:00,  7.69it/s]


In [16]:
import time
import pandas as pd

In [4]:
start = time.time()
train = pd.read_csv('../train.csv', index_col = 0)
test = pd.read_csv('../test.csv', index_col = 0)
print(time.time() - start)

73.76222538948059


In [17]:
train.shape, test.shape

((24780, 5122), (21540, 5121))

## 데이터 분리 (train, val)

In [43]:
train['label'].value_counts() / train['label'].value_counts().sum()

110    0.029021
17     0.025393
118    0.024184
114    0.024184
113    0.022975
         ...   
137    0.001209
153    0.001209
185    0.001209
42     0.001209
135    0.001209
Name: label, Length: 198, dtype: float64

### 랜덤하게 train test split

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train = train.drop(['label'], axis=1)
y_train = train['label']

In [8]:
X_trn, X_val, y_trn, y_val = train_test_split(X_train, y_train, test_size = 0.2, shuffle = True, random_state = 0)

In [9]:
X_trn.shape, X_val.shape, y_trn.shape, y_val.shape

((19848, 5121), (4962, 5121), (19848,), (4962,))

In [120]:
from sklearn.ensemble import RandomForestClassifier

In [121]:
def train_model(X, y):
    
    model = RandomForestClassifier(random_state=0, verbose=1, n_jobs=-1)
    model.fit(X, y)
    
    return model

In [122]:
model = train_model(X_trn, y_trn)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   48.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.0min finished


In [123]:
y_pred = model.predict_proba(X_val)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.5s finished


In [131]:
from sklearn.preprocessing import label_binarize
y_val_binarize = label_binarize(y_val, classes = range(198))

In [125]:
y_val.values

array([ 67,  68,  83, ...,  28,  68, 193], dtype=int64)

In [133]:
y_val_binarize[-1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [134]:
log_loss(y_val_binarize, y_pred)

0.8721437176576072

In [137]:
train['label'].value_counts() / train['label'].value_counts().sum()

110    0.029021
17     0.025393
118    0.024184
114    0.024184
113    0.022975
         ...   
137    0.001209
153    0.001209
185    0.001209
42     0.001209
135    0.001209
Name: label, Length: 198, dtype: float64

In [138]:
y_trn.value_counts()

110    570
17     511
118    483
114    472
117    471
      ... 
146     21
49      20
36      20
140     20
44      19
Name: label, Length: 198, dtype: int64

In [139]:
y_trn.value_counts()[[137, 153, 42]]

137    26
153    25
42     21
Name: label, dtype: int64

다행히도 198개의 라벨이 모두 있습니다.

### Stratified K fold

In [140]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)

In [153]:
loss_list = []

for trn_idx, val_idx in skf.split(X_train, y_train):
    
    X_trn, X_val, y_trn, y_val = X_train.iloc[trn_idx], X_train.iloc[val_idx], y_train.iloc[trn_idx], y_train.iloc[val_idx]
    trained_model = train_model(X_trn, y_trn)
    y_pred = trained_model.predict_proba(X_val)
    y_val_binarize = label_binarize(y_val, classes = range(198))
    loss = log_loss(y_val_binarize, y_pred)
    print(loss)
    loss_list.append(loss)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.4min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished


2.676753414529926


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.8s finished


1.3037120563564033


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   37.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished


1.5624730298106417


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   38.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


1.166969298265241


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   37.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished


1.8046789752691896


In [156]:
print(loss_list)
print(np.mean(loss_list))

[2.676753414529926, 1.3037120563564033, 1.5624730298106417, 1.166969298265241, 1.8046789752691896]
1.7029173548462804


In [144]:
y_train.value_counts()

110    720
17     630
118    600
114    600
113    570
      ... 
137     30
153     30
185     30
42      30
135     30
Name: label, Length: 198, dtype: int64

In [147]:
720 * 0.8

576.0

In [143]:
y_trn.value_counts()

110    576
17     504
118    480
114    480
113    456
      ... 
137     24
153     24
185     24
42      24
135     24
Name: label, Length: 198, dtype: int64

In [146]:
y_val.value_counts()

110    144
17     126
118    120
114    120
117    114
      ... 
45       6
49       6
98       6
57       6
195      6
Name: label, Length: 198, dtype: int64

# LightGBM

- Boosting은 여러개의 트리(혹은 다른 모델)를 만들되, 랜덤포레스트나 배깅과 같은 기법과는 다른게 기존에 있는 예측기를 조금씩 발전시켜서 이를 합한다는 개념이다.

- 랜덤포레스트는 병렬로 무지막지하게 많은 다양한 결정트리를 동시에 만든다면 부스팅은 점진적으로 Decision Tree를 발전시킨뒤에 이를 통합하는 과정을 거친다고 보면된다. 부스팅은 보통 두가지 방향이 있다.

- 정답지와 오답지간의 차이를 훈련에 다시 투입하여 gradient를 적극 이용해서 모델을 개선하는 방식. XGboost나 lightGBM이 여기에 속한다.

- GBM이란? : Gradient Boosting Machine(GBM)은 틀린부분에 가중치를 더하면서 진행하는 알고리즘

- LGBM은 머신러닝 부스팅 알고리즘의 성능을 높인것이라고 할 수 있다.

- 검증셋을 활용해서 검증셋에 대한 결과가 좋아지지 않을때까지 의사결정트리를 추가한다.

In [3]:
!pip install lightgbm

  Using cached lightgbm-3.1.0-py2.py3-none-win_amd64.whl (751 kB)


In [4]:
import lightgbm as lgb

In [5]:
import random
random.seed(0) # 어느 컴퓨터에서 추출해도 똑같은 값이 나오도록 고정
idx = random.sample(range(5121), 50) # 5000개의 컬럼중에 랜덤하게 50개를 추출

In [20]:
from sklearn.model_selection import StratifiedKFold 
# 클래스 개수가 불균등하게 분포가 되어있기 때문에 train이랑 validation을 나눌때 균등하게 나누기 위해 StratifiedKFold를 사용
skf = StratifiedKFold(n_splits=5) # n_splits=5 : 8대2로 나눔

for trn_idx, val_idx in skf.split(X_train, y_train):
    
    X_trn, X_val, y_trn, y_val = X_train.iloc[trn_idx], X_train.iloc[val_idx], y_train.iloc[trn_idx], y_train.iloc[val_idx]
    
    break # break -> for문을 다 돌리지 않고 1번만 돌림.

In [21]:
model_lgb = lgb.LGBMClassifier(n_estimators=50, learning_rate=0.03) 
# 분류문제이므로LGBMClassifier를 가져온다.
# learning_rate, n_estimators 등 이런것들이 사용자가 직접 설정해야하는 옵션인 '하이퍼파라미터' 

In [22]:
start = time.time()
model_lgb.fit(X_trn.iloc[:,idx], y_trn)# 여기서는 validaion셋을 넣지 않아서 n_estimators=50을 모두 학습한다. -> 안좋음. #fit, predict함수는 그대로 scikit-learn처럼 사용, #idx -> 위에서 랜덤하게 뽑은 50개의 칼럼만 가지고
time.time() - start

26.938925981521606

- early stopping
- 너무 적게 학습하면 즉 의사결정트리 수 n_estimators를 조금 주면 학습이 덜 됨 : 과소적합
- 너무 많이 학습하면 noise까지 학습해버리기 때문에 더 이상한 모델이 나옴. 새로 들어오는 test데이터에 잘 맞아야 하는데 train 데이터에만 너무 학습되어서 망함 : 과대적합

- **과소적합과 과대적합**

![](https://datascience.foundation/img/pdf_images/underfitting_and_overfitting_in_machine_learning_some_degrees_and_train_our_model.png)

- **검증 셋 활용**

![](https://datavedas.com/wp-content/uploads/2018/04/image003.jpg)




- **Training Error vs Validation Error**

  - 주황색 선이 validation error라고 봐주시길 바랍니다.

![](https://www.textbook.ds100.org/_images/bias_cv_train_test_error.png)

In [23]:
lgb_pred = model_lgb.predict_proba(X_val.iloc[:,idx])

In [24]:
model_lgb = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.03, objective='multiclass')

In [25]:
len(idx) 
#-> 학습 시간 줄이려고(너무 오래걸리므로 just for example)

50

In [26]:
start = time.time()
model_lgb.fit(X_trn.iloc[:,idx], y_trn, 
              eval_set=(X_val.iloc[:,idx], y_val), 
              verbose = 10, #verbose = 10 : 10번 반복할 때마다 logloss값을 보여준다.
              early_stopping_rounds=10) # early_stopping_rounds은 범위를 잡는 것, early_stopping_rounds를 조정해서 validation error가 최소가 되는 지점에 학습을 멈추도록
time.time() - start 
# validation 셋을 활용해서 원하는 지점까지 학습 함.
# n_estimators=100으로 설정해서 10, 20, 30.. 가다가 40~50사이에 loss값이 증가해서 42지점에서 알아서 멈춤. 
# 즉 validaion의 loss값이 최소가 될때 stop
# 이런식으로 하이퍼파라미터를 조정할 수도 있음.

Training until validation scores don't improve for 10 rounds
[10]	valid_0's multi_logloss: 4.26184
[20]	valid_0's multi_logloss: 4.14101
[30]	valid_0's multi_logloss: 4.08661
[40]	valid_0's multi_logloss: 4.0672
[50]	valid_0's multi_logloss: 4.07214
Early stopping, best iteration is:
[42]	valid_0's multi_logloss: 4.06639


32.25992274284363